In [ ]:
import numpy as np
import time

from opt_einsum import contract
import opt_einsum as oe

# 设置BLAS线程数

try:
    import mkl
    mkl.set_num_threads(1)
except ImportError:
    pass
try:
    import blas
    blas.set_num_threads(1)
except ImportError:
    pass

In [ ]:
NOCC = 20
NVIR = 80
NAO = NOCC + NVIR
NLAPLACE = 8

In [ ]:
cmoao_nocc = np.random.rand(NAO, NOCC)
cmoao_nvir = np.random.rand(NAO, NVIR)
tau_occ = np.random.rand(NOCC, NLAPLACE)
tau_vir = np.random.rand(NVIR, NLAPLACE)
orb_ene = np.random.rand(NAO)

eri = np.random.rand(NAO, NAO, NAO, NAO)

In [ ]:
# standard impl of MP2

cpu_begin = time.process_time()

# step 1. traint 

I_iajb = contract('pi,qa,pqrs,rj,sb->iajb',cmoao_nocc, cmoao_nvir, eri, cmoao_nocc, cmoao_nvir, optimize='optimal')

# step 2. build eijab

eia = orb_ene[:NOCC, None] - orb_ene[None, NOCC:]
eijab = 1.0 / (eia[:, :, None, None] + eia[None, None, :, :])

# step 3. build intermediate

intermediate = 2 * I_iajb - I_iajb.transpose(0, 3, 2, 1)

# step 4. get e2

e2 = contract('iajb,iajb,iajb->', intermediate, I_iajb, eijab, optimize='optimal')

cpu_end = time.process_time()

print("cputime = %.2f s" % (cpu_end - cpu_begin))


In [ ]:
# laplace impl of MP2

cpu_begin = time.process_time()

# step 1. traint 

I_iajb = contract('pi,qa,pqrs,rj,sb->iajb',cmoao_nocc, cmoao_nvir, eri, cmoao_nocc, cmoao_nvir, optimize='optimal')

# step 2. build intermediate

intermediate = 2 * I_iajb - I_iajb.transpose(0, 3, 2, 1)

# step 3. final 

path_info = oe.contract_path('iajb,iajb,ic,ac,jc,bc->', I_iajb, intermediate, tau_occ, tau_vir, tau_occ, tau_vir, optimize='optimal')
print(path_info[1])

e2 = contract('iajb,iajb,ic,ac,jc,bc->', I_iajb, intermediate, tau_occ, tau_vir, tau_occ, tau_vir, optimize='optimal')

cpu_end = time.process_time()

print("cputime = %.2f s" % (cpu_end - cpu_begin))